In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def leer_archivo(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    dimension = 0
    pesos_aristas = []
    reading_weights = False
    
    for line in lines:
        if line.startswith("DIMENSION"):
            dimension = int(line.split()[-1])
        elif line.startswith("EDGE_WEIGHT_SECTION"):
            reading_weights = True
            continue
        elif line.startswith("EOF"):
            break
        elif reading_weights:
            pesos_aristas.extend(map(int, line.split()))
            
    return dimension, pesos_aristas

In [3]:
def crear_grafo_ATSP(dimension, pesos_aristas):

    G = nx.DiGraph()

    index = 0
    for i in range(dimension):
        for j in range(dimension):
            if i != j:
                weight = pesos_aristas[index]
                G.add_edge(i, j, weight=weight)
            index += 1

    return G

In [4]:
def visualizar_grafo(G):
    pos = nx.spring_layout(G)  # positions for all nodes
    edge_labels = {(u, v): data['weight'] for u, v, data in G.edges(data=True)}
    
    plt.figure(figsize=(10, 10))
    
    nx.draw(G, pos, with_labels=True, node_size=700, node_color='skyblue', font_size=10, font_weight='bold', arrows=True)
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')
    
    plt.title("Graph Visualization of ATSP")
    plt.show()

In [5]:
def calcular_costo(G, solucion):
    total_cost = 0
    
    for i in range(len(solucion) - 1):
        total_cost += G[solucion[i]][solucion[i+1]]['weight']
        
    total_cost += G[solucion[-1]][solucion[0]]['weight']
    
    return total_cost

In [6]:
def visualize_hamiltonian_circuit(G, circuit):
    pos = circular_layout_based_on_circuit(circuit)
    plt.figure(figsize=(5, 5))
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, nodelist=circuit, node_size=700, node_color='skyblue')

    # Draw only the edges in the circuit
    edges = [(circuit[i], circuit[i + 1]) for i in range(len(circuit) - 1)]
    edges.append((circuit[-1], circuit[0]))  # Close the circuit
    
    edge_labels = {(u, v): G[u][v]['weight'] for u, v in edges}
    
    
    nx.draw_networkx_edges(G, pos, edgelist=edges, edge_color='blue', width=2, arrows=True)
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')
    
    # Draw node labels
    nx.draw_networkx_labels(G, pos, font_size=10, font_weight='bold')
    
    plt.title("Hamiltonian Circuit Visualization")
    plt.show()

def circular_layout_based_on_circuit(circuit):
    num_nodes = len(circuit)
    angle_step = 2 * np.pi / num_nodes
    pos = {}
    for i, node in enumerate(circuit):
        angle = i * angle_step
        pos[node] = np.array([np.cos(angle), np.sin(angle)])
    return pos

In [7]:
def distancia_promedio_de_vecinos(G, nodo):
    vecinos = list(G.neighbors(nodo))
    
    if not vecinos:
        return float('inf')  # If a node has no neighbors, return infinity
    total_distance = sum(nx.shortest_path_length(G, nodo, neighbor) for neighbor in vecinos)
    return total_distance / len(vecinos)

In [8]:
def heuristica_1(G):
    start_node = 0
    visited = [start_node]
    current_node = start_node

    while len(visited) < len(G.nodes):
        next_node = min(
            (node for node in G.neighbors(current_node) if node not in visited),
            key=lambda node: G[current_node][node]['weight'],
            default=None
        )
        if next_node is None:
            break
        
        visited.append(next_node)
        current_node = next_node
    
    # Return to the start node to complete the circuit
    total_cost = calcular_costo(G, visited)
    
    return visited, total_cost

In [9]:
def heuristica_2(G):
    
    visited = [0]
    current_node = 0
    
    while len(visited) < len(G.nodes):
        neighbors = set(G.neighbors(current_node)) - set(visited)
        if not neighbors:
            break
        
        best_average = float('inf')
        next_node = None
        
        for node in neighbors:
            if node not in visited:
                average = distancia_promedio_de_vecinos(G, node)
                if average < best_average:
                    best_average = average
                    next_node = node
        
        visited.append(next_node)
        current_node = next_node
    
    total_cost = calcular_costo(G, visited)
    
    return visited,total_cost

In [10]:
def heuristica_3(G):
    
    start_node = 0
    visited = [start_node]
    current_node = start_node

    while len(visited) < len(G.nodes):
        next_node = min(
            (node for node in G.neighbors(current_node) if node not in visited),
            key=lambda node: G[node][current_node]['weight'],
            default=None
        )
        if next_node is None:
            break
        
        visited.append(next_node)
        current_node = next_node
    
    total_cost = calcular_costo(G, visited)
    
    return visited, total_cost

In [11]:
def operador_swap(solucion):

    neighborhood = []

    # Get all pairs (i, j) where i != j
    for i in range(len(solucion)):
        for j in range(i + 1, len(solucion)):
            # Swap the nodes at position i and j
            nueva_solucion = solucion[:]
            nueva_solucion[i], nueva_solucion[j] = nueva_solucion[j], nueva_solucion[i]
            neighborhood.append(nueva_solucion)

    return neighborhood

In [12]:
def operador_insert(solucion):

    neighborhood = []

    # Iterate over all nodes in the circuit
    for i in range(len(solucion)):
        # Remove node from its current position
        nodo = solucion[i]
        solucion_temp = solucion[:i] + solucion[i+1:]

        # Insert node in every possible position in the reduced circuit
        for j in range(len(solucion_temp) + 1):
            nueva_solucion = solucion_temp[:j] + [nodo] + solucion_temp[j:]
            
            neighborhood.append(nueva_solucion)

    return neighborhood

In [76]:
def operador_2opt(solucion):
    
    neighborhood = []
    print(solucion)
    for i in range(1, len(solucion) - 2):
        for k in range(i + 1, len(solucion) - 1):
            new_solution = solucion[:i] + solucion[i:k+1][::-1] + solucion[k+1:]
            print(new_solution)
            neighborhood.append(new_solution)
    
    return neighborhood

In [58]:
def busqueda_local(G, solucion, operador:str):

        print(f'Iniciando busqueda local con operador {operador}')
        
        best_solution = solucion
        best_cost = calcular_costo(G, best_solution)

        i:int = 0
        
        while i < 1000:
            print(f"Iteration {i} - Best cost: {best_cost}")
            if operador == "swap":
                neighborhood = operador_swap(best_solution)
            elif operador == "insert":
                neighborhood = operador_insert(best_solution)
            elif operador == "2opt":
                neighborhood = operador_2opt(best_solution)
                
            improved = False

            if operador == "all":
                
                neighborhood_2opt = operador_2opt(best_solution)
                for neighbor in neighborhood_2opt:
                    cost = calcular_costo(G, neighbor)
                    if cost < best_cost:
                        best_solution = neighbor
                        best_cost = cost
                        improved = True           
                
                neighborhood_swap = operador_swap(best_solution)
                for neighbor in neighborhood_swap:
                    cost = calcular_costo(G, neighbor)
                    if cost < best_cost:
                        best_solution = neighbor
                        best_cost = cost
                        improved = True
                
                neighborhood_insert = operador_insert(best_solution)
                for neighbor in neighborhood_insert:
                    cost = calcular_costo(G, neighbor)
                    if cost < best_cost:
                        best_solution = neighbor
                        best_cost = cost
                        improved = True            
                
            
            else:
                for neighbor in neighborhood:
                    cost = calcular_costo(G, neighbor)
                    
                    if cost < best_cost:
                        best_solution = neighbor
                        best_cost = cost
                        improved = True
    
            i+=1
            if not improved:
                break
            
        return best_solution, best_cost

In [78]:
# Example usage
filename = './instancias/rbg443.atsp'
dimension, pesos_aristas = leer_archivo(filename)

G = crear_grafo_ATSP(dimension, pesos_aristas)



In [79]:
CH_heuristica1,total_cost = heuristica_1(G)
CH_heuristica2,total_cost2 = heuristica_2(G)
CH_heuristica3,total_cost3 = heuristica_3(G)

print("Costo total 1: ", total_cost)  
print(CH_heuristica1)

print("Costo total 2: ", total_cost2)  
print(CH_heuristica2)

print("Costo total 3: ", total_cost3)  
print(CH_heuristica3)

In [77]:
print("Solucion inicial: ", CH_heuristica1)
print("Costo incial: ", total_cost)

solucion_mejorada_insert = busqueda_local(G, CH_heuristica1, "insert")

print("Solucion mejorada con insert: ", solucion_mejorada_insert[0])
print("Costo total: ", solucion_mejorada_insert[1])

solucion_mejorada_swap = busqueda_local(G, CH_heuristica1, "swap")

print("Solucion mejorada con swap: ", solucion_mejorada_swap[0])
print("Costo total: ", solucion_mejorada_swap[1])

solucion_mejorada_2opt = busqueda_local(G, CH_heuristica1, "2opt")

print("Solucion mejorada con 2opt: ", solucion_mejorada_2opt[0])
print("Costo total: ", solucion_mejorada_2opt[1])

Solucion inicial:  [0, 11, 1, 9, 10, 12, 2, 13, 7, 8, 16, 5, 6, 14, 15, 3, 4]
Costo incial:  92
Iniciando busqueda local con operador insert
Iteration 0 - Best cost: 92
Iteration 1 - Best cost: 58
Iteration 2 - Best cost: 42
Solucion mejorada con insert:  [5, 0, 11, 9, 10, 12, 2, 13, 1, 7, 8, 16, 6, 14, 15, 3, 4]
Costo total:  42
Iniciando busqueda local con operador swap
Iteration 0 - Best cost: 92
Iteration 1 - Best cost: 47
Iteration 2 - Best cost: 44
Iteration 3 - Best cost: 42
Solucion mejorada con swap:  [5, 11, 0, 9, 10, 12, 2, 13, 1, 8, 16, 7, 6, 14, 15, 3, 4]
Costo total:  42
Iniciando busqueda local con operador 2opt
Iteration 0 - Best cost: 92
[0, 11, 1, 9, 10, 12, 2, 13, 7, 8, 16, 5, 6, 14, 15, 3, 4]
[0, 1, 11, 9, 10, 12, 2, 13, 7, 8, 16, 5, 6, 14, 15, 3, 4]
[0, 9, 1, 11, 10, 12, 2, 13, 7, 8, 16, 5, 6, 14, 15, 3, 4]
[0, 10, 9, 1, 11, 12, 2, 13, 7, 8, 16, 5, 6, 14, 15, 3, 4]
[0, 12, 10, 9, 1, 11, 2, 13, 7, 8, 16, 5, 6, 14, 15, 3, 4]
[0, 2, 12, 10, 9, 1, 11, 13, 7, 8, 16, 5, 

In [71]:
print("Solucion inicial: ", CH_heuristica2)
print("Costo incial: ", total_cost2)

solucion_mejorada_insert = busqueda_local(G, CH_heuristica2, "insert")

print("Solucion mejorada con insert: ", solucion_mejorada_insert[0])
print("Costo total: ", solucion_mejorada_insert[1])

solucion_mejorada_swap = busqueda_local(G, CH_heuristica2, "swap")

print("Solucion mejorada con swap: ", solucion_mejorada_swap[0])
print("Costo total: ", solucion_mejorada_swap[1])

solucion_mejorada_2opt = busqueda_local(G, CH_heuristica2, "2opt")

print("Solucion mejorada con 2opt: ", solucion_mejorada_2opt[0])
print("Costo total: ", solucion_mejorada_2opt[1])

Solucion inicial:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 13, 14, 15]
Costo incial:  190
Iniciando busqueda local con operador insert
Iteration 0 - Best cost: 190
Iteration 1 - Best cost: 124
Iteration 2 - Best cost: 83
Iteration 3 - Best cost: 63
Iteration 4 - Best cost: 60
Solucion mejorada con insert:  [1, 13, 2, 0, 11, 16, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 15]
Costo total:  60
Iniciando busqueda local con operador swap
Iteration 0 - Best cost: 190
Iteration 1 - Best cost: 104
Iteration 2 - Best cost: 66
Iteration 3 - Best cost: 60
Iteration 4 - Best cost: 55
Iteration 5 - Best cost: 41
Iteration 6 - Best cost: 39
Solucion mejorada con swap:  [14, 15, 6, 5, 4, 3, 16, 7, 8, 9, 10, 1, 12, 2, 13, 0, 11]
Costo total:  39
Iniciando busqueda local con operador 2opt
Iteration 0 - Best cost: 190
Iteration 1 - Best cost: 74
Iteration 2 - Best cost: 48
Iteration 3 - Best cost: 42
Iteration 4 - Best cost: 40
Iteration 5 - Best cost: 39
Solucion mejorada con 2opt:  [0, 11, 2, 13, 9, 10, 

In [72]:
print("Solucion inicial: ", CH_heuristica3)
print("Costo incial: ", total_cost3)

solucion_mejorada_insert = busqueda_local(G, CH_heuristica3, "insert")

print("Solucion mejorada con insert: ", solucion_mejorada_insert[0])
print("Costo total: ", solucion_mejorada_insert[1])

solucion_mejorada_swap = busqueda_local(G, CH_heuristica3, "swap")

print("Solucion mejorada con swap: ", solucion_mejorada_swap[0])
print("Costo total: ", solucion_mejorada_swap[1])

solucion_mejorada_2opt = busqueda_local(G, CH_heuristica3, "2opt")

print("Solucion mejorada con 2opt: ", solucion_mejorada_2opt[0])
print("Costo total: ", solucion_mejorada_2opt[1])

Solucion inicial:  [0, 11, 1, 9, 10, 12, 2, 13, 7, 8, 16, 5, 6, 14, 15, 3, 4]
Costo incial:  92
Iniciando busqueda local con operador insert
Iteration 0 - Best cost: 92
Iteration 1 - Best cost: 58
Iteration 2 - Best cost: 42
Solucion mejorada con insert:  [5, 0, 11, 9, 10, 12, 2, 13, 1, 7, 8, 16, 6, 14, 15, 3, 4]
Costo total:  42
Iniciando busqueda local con operador swap
Iteration 0 - Best cost: 92
Iteration 1 - Best cost: 47
Iteration 2 - Best cost: 44
Iteration 3 - Best cost: 42
Solucion mejorada con swap:  [5, 11, 0, 9, 10, 12, 2, 13, 1, 8, 16, 7, 6, 14, 15, 3, 4]
Costo total:  42
Iniciando busqueda local con operador 2opt
Iteration 0 - Best cost: 92
Iteration 1 - Best cost: 75
Solucion mejorada con 2opt:  [0, 11, 13, 2, 12, 10, 9, 1, 7, 8, 16, 5, 6, 14, 15, 3, 4]
Costo total:  75


In [73]:
print("Solucion inicial: ", CH_heuristica1)
print("Costo incial: ", total_cost)

solucion_mejorada_all = busqueda_local(G, CH_heuristica1, "all")

print("Solucion mejorada con todos: ", solucion_mejorada_all[0])
print("Costo total: ", solucion_mejorada_all[1])

Solucion inicial:  [0, 11, 1, 9, 10, 12, 2, 13, 7, 8, 16, 5, 6, 14, 15, 3, 4]
Costo incial:  92
Iniciando busqueda local con operador all
Iteration 0 - Best cost: 92
Iteration 1 - Best cost: 41
Iteration 2 - Best cost: 39
Solucion mejorada con todos:  [0, 11, 13, 2, 12, 10, 9, 1, 7, 8, 16, 3, 4, 14, 6, 5, 15]
Costo total:  39


In [74]:
print("Solucion inicial: ", CH_heuristica2)
print("Costo incial: ", total_cost2)

solucion_mejorada_all2 = busqueda_local(G, CH_heuristica2, "all")

print("Solucion mejorada con todos: ", solucion_mejorada_all2[0])
print("Costo total: ", solucion_mejorada_all2[1])

Solucion inicial:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 13, 14, 15]
Costo incial:  190
Iniciando busqueda local con operador all
Iteration 0 - Best cost: 190
Iteration 1 - Best cost: 42
Iteration 2 - Best cost: 39
Solucion mejorada con todos:  [11, 0, 13, 2, 1, 9, 12, 10, 16, 8, 7, 3, 4, 5, 6, 14, 15]
Costo total:  39


In [75]:
print("Solucion inicial: ", CH_heuristica3)
print("Costo incial: ", total_cost3)

solucion_mejorada_all3 = busqueda_local(G, CH_heuristica3, "all")

print("Solucion mejorada con todos: ", solucion_mejorada_all3[0])
print("Costo total: ", solucion_mejorada_all3[1])

Solucion inicial:  [0, 11, 1, 9, 10, 12, 2, 13, 7, 8, 16, 5, 6, 14, 15, 3, 4]
Costo incial:  92
Iniciando busqueda local con operador all
Iteration 0 - Best cost: 92
Iteration 1 - Best cost: 41
Iteration 2 - Best cost: 39
Solucion mejorada con todos:  [0, 11, 13, 2, 12, 10, 9, 1, 7, 8, 16, 3, 4, 14, 6, 5, 15]
Costo total:  39
